In [ ]:
import uproot as uproot3
import ROOT

In [ ]:
from glob import glob

Scaling factors are stored in `fHistXsection`, which is TProfile object.  
It makes obtaining its values using `uproot` problematic as its does not have fields containing values which are shown after plotting.

They can be however calculated based on other fields as `hist._fTsumwy / hist._fTsumw`   
which agrees with suggestion here: https://stackoverflow.com/questions/58541426/loading-tprofile-in-uproot-issue-getting-y-values  
and with $\texttt{TProfile}$ docs: https://root.cern.ch/doc/master/classTProfile.html#ada88934586554394904cf4b85d8ce39e

The values however slightly differ from what is plotted in ROOT (`h->GetBinContent(i)`) (differences ~ 0.4-0.7%)   


for **LHC16h3** the **calculated** values are:  

     'ptbin1': 16.054692265163954,
     'ptbin2': 4.599556358210402,
     'ptbin3': 2.1449183859321876,
     'ptbin4': 0.7804427076089806,
     'ptbin5': 0.26417809194471015,
     'ptbin6': 0.09738198949503354,
     'ptbin7': 0.02920915652260396,
     'ptbin8': 0.009867379547080737,
     'ptbin9': 0.0040377202366385515,
     'ptbin10': 0.001350376192733647,
     'ptbin11': 0.0005286526390493335,
     'ptbin12': 0.00018789783969057545,
     'ptbin13': 9.194764462105562e-05,
     'ptbin14': 4.2784775394613205e-05,
     'ptbin15': 2.0871151551456155e-05
 
 while plotting in **ROOT** shows:
 
    'ptbin1' :16.14,
    'ptbin2' : 4.626,
    'ptbin3' : 2.158,
    'ptbin4' : 0.78583,
    'ptbin5' : 0.26581,
    'ptbin6' : 0.097988,
    'ptbin7' : 0.0294,
    'ptbin8' : 0.009909,
    'ptbin9' : 0.0040611,
    'ptbin10': 0.0013595,
    'ptbin11': 0.00053207,
    'ptbin12': 0.00018912,
    'ptbin13': 0.00009255,
    'ptbin14': 0.00004305,
    'ptbin15': 0.00002100,

which corresponds to **differeces** (no abs taken so the differences are consistently higher/lower):

    ptbin1 0.53%
    ptbin2 0.57%
    ptbin3 0.61%
    ptbin4 0.69%
    ptbin5 0.61%
    ptbin6 0.62%
    ptbin7 0.65%
    ptbin8 0.42%
    ptbin9 0.58%
    ptbin10 0.67%
    ptbin11 0.64%
    ptbin12 0.65%
    ptbin13 0.65%
    ptbin14 0.62%
    ptbin15 0.61%

## obtain scaling factors

In [ ]:
# HARD-CODED from fHistXsection values for given ptbins
#  ------- LHC16h3 ----------
scaling_factor = {
    'ptbin1' :16.14,
    'ptbin2' : 4.626,
    'ptbin3' : 2.158,
    'ptbin4' : 0.78583,
    'ptbin5' : 0.26581,
    'ptbin6' : 0.097988,
    'ptbin7' : 0.0294,
    'ptbin8' : 0.009909,
    'ptbin9' : 0.0040611,
    'ptbin10': 0.0013595,
    'ptbin11': 0.00053207,
    'ptbin12': 0.00018912,
    'ptbin13': 0.00009255,
    'ptbin14': 0.00004305,
    'ptbin15': 0.00002100,
}

In [ ]:
def read_event_hist(fpath, jet_flavour, hist_name='fHistXsection'):
    hist_dir_name = f'AliAnalysisTaskJetExtractor_Jet_AKTChargedR040_tracks_pT0150_E_scheme_{jet_flavour}Jets_histos'
    froot = uproot3.open(fpath)
    hist_dir = froot['ChargedJetsHadronCF'][hist_dir_name]
    hist = [h for h in  hist_dir if hist_name in str(h.name)][0]
    return hist._fTsumwy/hist._fTsumw


scaling_factor_calc = dict()
for fpath in sorted(glob('../ana_results/iter3/LHC16h3/ptbin*/AnalysisResults.root'), key=lambda x: -int(x.split('/')[-2][5:]))[::-1]:
    ptbin = fpath.split('/')[-2]
    scaling_factor_calc[ptbin] = read_event_hist(fpath, 'udsg', 'fHistXsection')
    
scaling_factor_calc

In [ ]:
# calculated relative differences (no abs!)
for ptbin in scaling_factor.keys():
    print(ptbin, f'{(scaling_factor[ptbin]-scaling_factor_calc[ptbin])/scaling_factor[ptbin]*100:.2f}%')

## plot $p_T$ of the initial hard process

In [ ]:
%jsroot on
c = ROOT.TCanvas("myCanvasName","The Canvas Title",800,400)

start = True
for fpath in sorted(glob('../ana_results/iter3/LHC16h3/ptbin*/AnalysisResults.root'), key=lambda x: -int(x.split('/')[-2][5:])):
    ptbin = fpath.split('/')[-2]    
    tree_name = 'JetTree_AliAnalysisTaskJetExtractor_Jet_AKTChargedR040_tracks_pT0150_E_scheme_udsgJets'
    froot = ROOT.TFile(fpath)
    tree = froot.Get(tree_name)
    h = ROOT.TH1D(f'h_{ptbin}', f'h_{ptbin}', 25000,0,250)
    tree.Draw(f"Event_PtHard >> h_{ptbin}", "", "goff")
    h.Scale(scaling_factor[ptbin])  # hardcoded
#     h.Scale(scaling_factor_calc[ptbin]) # calculated
    if start: 
        h.SetMaximum(1e6)
        h.DrawCopy("HIST")
    else: 
        h.DrawCopy("HIST,SAME")
    start = False

c.SetLogy()
c.Draw()

## plot $p_T$ of the jets

_large jumps come from single large-pt jet created in low hard pt bin and therefore scaled by huge factor_

In [ ]:
%jsroot on
c = ROOT.TCanvas("myCanvasName","The Canvas Title",800,400)

start = True
for fpath in sorted(glob('../ana_results/iter3/LHC16h3/ptbin*/AnalysisResults.root'), key=lambda x: -int(x.split('/')[-2][5:])):
    ptbin = fpath.split('/')[-2]
    print(fpath,ptbin)
    
    tree_name = 'JetTree_AliAnalysisTaskJetExtractor_Jet_AKTChargedR040_tracks_pT0150_E_scheme_udsgJets'
    froot = ROOT.TFile(fpath)
    tree = froot.Get(tree_name)
    h = ROOT.TH1D(f'h_{ptbin}', f'h_{ptbin}', 200,0,200)
    tree.Draw(f"Jet_Pt >> h_{ptbin}", "", "goff")
    h.Scale(scaling_factor[ptbin])  # hardcoded
#     h.Scale(scaling_factor_calc[ptbin]) # calculated
    if start: 
        h.SetMaximum(1e8)
        h_total = h.Clone()
        h_total.SetDirectory(0) # prevents from deleting it from the scope after closing currently opened file
        h_total.SetTitle('h_total')
        h_total.SetName('h_total')
#         h.DrawCopy("E0")
    else: 
        h_total.Add(h)
#         h.DrawCopy("E0,SAME")
    start = False
h_total.Draw("E0, SAME")
c.SetLogy()
c.Draw()

same **split** by hard pt bin

In [ ]:
%jsroot on
c = ROOT.TCanvas("myCanvasName","The Canvas Title",800,400)

start = True
for fpath in sorted(glob('../ana_results/iter3/LHC16h3/ptbin*/AnalysisResults.root'), key=lambda x: -int(x.split('/')[-2][5:])):
    ptbin = fpath.split('/')[-2]
    print(fpath,ptbin)
    
    tree_name = 'JetTree_AliAnalysisTaskJetExtractor_Jet_AKTChargedR040_tracks_pT0150_E_scheme_udsgJets'
    froot = ROOT.TFile(fpath)
    tree = froot.Get(tree_name)
    h = ROOT.TH1D(f'h_{ptbin}', f'h_{ptbin}', 100,0,200)
    tree.Draw(f"Jet_Pt >> h_{ptbin}", "", "goff")
    h.Scale(scaling_factor[ptbin])  # hardcoded
#     h.Scale(scaling_factor_calc[ptbin]) # calculated
    if start: 
        h.SetMaximum(1e7)
        h.DrawCopy("HIST")
    else: 
        h.DrawCopy("HIST,SAME")
    start = False

c.SetLogy()
c.Draw()